# Tarea 2

**Envío del PR inicial: 07.09.2018**

**Aceptación del PR: 14.09.2018**

In [1]:
using Plots

In [2]:
# Defino un tipo parametrico
struct Dual{T, K}
    x::T
    x′::K
end

In [3]:
Dual(c::Real) = Dual(c, 0.)
dual(x0::Real) = Dual(x0, 1.)

dual (generic function with 1 method)

In [4]:
import Base: +, -, *, /, ^

In [5]:
+(u::Dual, v::Dual) = Dual(u.x + v.x, u.x′ + v.x′)
-(u::Dual, v::Dual) = Dual(u.x - v.x, u.x′ - v.x′)
*(u::Dual, v::Dual) = Dual( u.x * v.x, u.x * v.x′ + u.x′ * v.x)

* (generic function with 220 methods)

In [6]:
function /(u::Dual, v::Dual)
    y = u.x / v.x
    Dual( y, (u.x′ - y * v.x′)/v.x )
end

/ (generic function with 90 methods)

In [7]:
function ^(u::Dual, n::Int)
    y = u.x^(n-1)
    Dual(u.x^n, n * y * u.x′)
end

^ (generic function with 57 methods)

## 1

Extiendan los operadores `+`, `-`, `*` y `/` para que funcionen cuando una de las dos entradas es un `Dual` y la otra es un `Real` (número cualquiera), sin importar el orden.

In [8]:
+(a::Real, u::Dual) = Dual(a + u.x, u.x′)
+(u::Dual, a::Real) = Dual(a + u.x, u.x′)

+ (generic function with 203 methods)

In [9]:
-(a::Real, u::Dual) = Dual(a - u.x, u.x′)
-(u::Dual, a::Real) = Dual(u.x - a, u.x′)

- (generic function with 225 methods)

In [10]:
*(a::Real, u::Dual) = Dual(a * u.x, u.x′ * a)
*(u::Dual, a::Real) = Dual(a * u.x, u.x′ * a)

* (generic function with 222 methods)

In [11]:
/(a::Real, u::Dual) = Dual(a / u.x, -((a / u.x)*u.x′) / u.x)
/(u::Dual, a::Real) = Dual(u.x / a, u.x′ / a)

/ (generic function with 92 methods)

## 2

Construyan 2 tests para cada una de las operaciones aritméticas definidas para los `Dual`es, incluyendo las que definieron en el ejercicio 1.

In [21]:
using Base.Test

## 3

Sobrecarguen la función `show` (que está en Base) para que cuando se impriman los duales aparezcan con la notación $\varepsilon$ que usamos al principio de este notebook, por ejemplo, el resultado de `Dual(1,2)` debe ser parecido a `1.0 + 2.0 ε`.

In [12]:
import Base:show

In [13]:
function show(io::IO, u::Dual)
    u.x′ > 0 ? print(u.x,"+",u.x′,"ε"): print(u.x, u.x′,"ε")
end

show (generic function with 319 methods)

In [14]:
dual(1.)

1.0+1.0ε

## 4

Obtengan la derivada de 
$$g(x) = \frac{3x^2-8x+5}{7x^3-1}$$
en $x_0=1$.

In [15]:
f(x) = 3*x^2 - 8*x + 5
g(x) = (7*x^3) - 1
h(x) = f(x) / g(x)

h (generic function with 1 method)

In [16]:
x = dual(1.)

1.0+1.0ε

In [17]:
h(x)

0.0-0.3333333333333333ε

In [18]:
# La derivada es lo siguiente
h′(x) = ((6*x - 8) * (7*x^3 - 1) - (3*x^2 - 8*x + 5) * (21*x^2 - 1)) / (7x^3 - 1)^2

h′ (generic function with 1 method)

Entonces la componente $\varepsilon$ de $h(x_D)$ con $x_D$ un dual tiene que ser igual a $h'(x)$, veamos que esto es cierto

In [22]:
@test h(x) == Dual(h(1.), h′(1.))

Test Passed

Éxito!!!

## 5

- Recordando la regla de la cadena(!!!), extiendan el usar los `Dual` a las funciones `sqrt`, `exp`, `log`, `sin`, `cos`, `sinh` y `cosh`. 

- Muestren que las cosas dan los resultados que esperan usando pruebas como hicieron en el ejercicio 2.

- Calculen la derivada de $h(x) = \sin\Big(x^3 - \frac{2}{x^6}\Big)$ en $x_0 = 2$. ¿Qué tan preciso es el resultado?(Pueden usar cualquier otra manera de obtener el resultado correcto, sólo tienen que ser claros en la explicación.)

- Dibujen, para $x_0 \in [1,5]$ la función $h'(x)$.

## 6

Implementen el método de Newton para una función arbitraria $f(x)$ en una variable, explotando el uso de los `Dual`. 

- Obtengan, usando su función para el método de Newton, las raices del polinomio de Wilkinson 

$$
W_{6}(x) = (x-1)(x-2)(x-3)(x-4)(x-5)(x-6),
$$ 

usando como iterados iniciales $x_0=2.2$ y $x_0=2.45$.
    
- Hagan lo mismo que en el inciso anterior para $g(x) = x \sin(2x)$, con $x_0=0.7$.

## 7

Argumenten qué podrían hacer para extender la idea de los `Dual` y calcular derivadas aún más altas. Como caso concreto, piensen en querer obtener la derivada 18 de funciones como las que hemos usado arriba